# EN ESTA NOTEBOOK SE IMPLEMENTARÁN LOS ALGORITMOS DE INTELIGENCIA ARTIFICIAL SUPERVISADOS Y SE EVALUARÁN LOS RESULTADOS PARA EL PROYECTO




## ARBOLES DE DECISIÓN

In [12]:
#importanción de librerías a utilizar

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

#ARBOL DE DECISIÓN
from sklearn.tree import DecisionTreeClassifier   #usqar arbol de decision
from sklearn.model_selection import train_test_split    #permite dividir dataset
from sklearn.metrics import accuracy_score  
from sklearn.preprocessing import StandardScaler #  librería para escalar las variables independientes


In [20]:
# IMPORTACIÓN DEL DATASET REDUCIDO LUEGO DEL PROCESAMIENTO DE COLUMNAS

df = pd.read_csv('friesgo_2018_reduc.csv',  sep=',', encoding='utf-8')

df.head(5)

,id,region,tamanio_aglomerado,tipo_vivienda,obtiene_gas,obtiene_agua,baño,cant_componentes,tipo_hogar,ingresos,...,considera_su_alimentacion,promedio_fv_diario,consumo_5porc_fv,colesterol_alto,consumido_alcohol,consumo_regular_riesgo,dijeron_diabetes,familiares_directos_diabetes,familiares_no_directos_diabetes,control_diabetes
0,1128639,1,1,3,1,1,1,2,2,50000,...,2,4.00,2,2,1,2.0,2,2,2,1
1,1709939,1,1,1,1,1,1,3,4,40000,...,2,6.43,1,1,1,2.0,2,2,2,1
2,6874130,1,1,3,1,1,1,2,2,40000,...,2,2.86,2,2,1,2.0,2,2,2,1
3,10319375,1,1,3,4,1,1,1,1,80000,...,2,1.43,2,1,1,2.0,2,2,1,1
4,11140857,1,1,3,1,1,1,2,8,30000,...,2,1.43,2,2,1,2.0,2,2,99,1


In [21]:
df.columns

Index(['id', 'region', 'tamanio_aglomerado', 'tipo_vivienda', 'obtiene_gas',
       'obtiene_agua', 'baño', 'cant_componentes', 'tipo_hogar', 'ingresos',
       'asignaciones', 'bhch02', 'sexo', 'edad', 'situacion_conyugal',
       'nivel_instruccion', 'cobertura_salud_nueva', 'ansiedad_depresion',
       'af_intensa_veces_por_semana', 'af_intensa_min_totales_por_semana',
       'af_intensa_min_totales_por_semana_nsnc',
       'af_moderada_veces_por_semana', 'af_moderada_min_totales_por_semana',
       'actividad_fisica_moderada_min_totales_por_semana_nsnc',
       'caminata_veces_por_semana', 'caminata_min_totales_por_semana',
       'caminata_min_totales_por_semana_nsnc', 'tiempo_sentado_min_por_dia',
       'tiempo_sentado_min_por_dia_nsnc',
       'razon_ninguna_actividad_la_ultima_semana',
       'razon_poca_actividad_la_ultima_semana', 'af_fue_actividad_domestica',
       'af_fue_actividad_laboral', 'af_fue_traslado', 'af_fue_deporte',
       'nivel_actividad_fisica', 'barreras_a

In [22]:
#muevo columnas de variables dependiendtes al final

cols_to_move = [ 'veces_presion_alta', 'colesterol_alto', 'dijeron_diabetes']

df = df[[col for col in df if col not in cols_to_move] + cols_to_move]

print (df.iloc[:,-4:-1]) 

       control_diabetes  veces_presion_alta  colesterol_alto
0                     1                   3                2
1                     1                   3                1
2                     1                   3                2
3                     1                   3                1
4                     1                   3                2
...                 ...                 ...              ...
26193                 1                   3                2
26194                 1                   2                1
26195                 1                   2                1
26196                 1                   3                2
26197                 1                   2                2

[26198 rows x 3 columns]


In [23]:
# Lista de las regiones únicas
regiones = df['region'].unique()
print(regiones)

[1 2 3 4 5 6]


In [24]:
# Crear un diccionario para almacenar los DataFrames por región
dfs_por_region = {}


In [25]:
# Iterar sobre cada región y guardar un nuevo DF por cada una

for region in regiones:
    # Filtrar los datos por la región actual
    df_region = df[df['region'] == region]
    # Guardar el DataFrame en el diccionario usando la región como clave
    dfs_por_region[region] = df_region


In [26]:
# Crear diccionarios para almacenar modelos y resultados por región
modelos_por_region = {}
resultados_por_region = {}

# Iterar sobre cada región para entrenar y evaluar modelos
for region in regiones:
    # Obtener el DataFrame de la región actual
    df_region = dfs_por_region[region]
    
    # Variables independientes
    X_region = df_region.iloc[:, 1:-3]  # Todas las columnas menos las últimas tres y la primera (id)
    
    # Variables dependientes
    y_region_colesterol = df_region['colesterol_alto']
    y_region_diabetes = df_region['dijeron_diabetes']
    y_region_hipertension = df_region['veces_presion_alta']

    # Dividir los datos en conjuntos de entrenamiento y prueba para cada enfermedad
    X_train_col, X_test_col, y_train_col, y_test_col = train_test_split(X_region, y_region_colesterol, test_size=0.25, random_state=0)
    X_train_dia, X_test_dia, y_train_dia, y_test_dia = train_test_split(X_region, y_region_diabetes, test_size=0.25, random_state=0)
    X_train_hip, X_test_hip, y_train_hip, y_test_hip = train_test_split(X_region, y_region_hipertension, test_size=0.25, random_state=0)
    
    # Crear y entrenar el modelo de árbol de decisión para cada enfermedad
    clf_col = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=4)
    clf_col.fit(X_train_col, y_train_col)
    
    clf_dia = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=4)
    clf_dia.fit(X_train_dia, y_train_dia)
    
    clf_hip = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=4)
    clf_hip.fit(X_train_hip, y_train_hip)
    
    # Probar modelo con conjunto de prueba
    y_pred_col = clf_col.predict(X_test_col)
    y_pred_dia = clf_dia.predict(X_test_dia)
    y_pred_hip = clf_hip.predict(X_test_hip)
    
    # Exactitud
    acc_col = accuracy_score(y_test_col, y_pred_col)
    acc_dia = accuracy_score(y_test_dia, y_pred_dia)
    acc_hip = accuracy_score(y_test_hip, y_pred_hip)
    
    # Almacenar los modelos y sus resultados por región
    modelos_por_region[region] = {
        'colesterol': clf_col,
        'diabetes': clf_dia,
        'hipertension': clf_hip
    }
    
    resultados_por_region[region] = {
        'colesterol': acc_col,
        'diabetes': acc_dia,
        'hipertension': acc_hip
    }
    
    
    
    

In [27]:
# Mostrar los resultados
for region, resultados in resultados_por_region.items():
    print(f"Región: {region}")
    print(f"Exactitud del modelo para colesterol: {resultados['colesterol']:.4f}")
    print(f"Exactitud del modelo para diabetes: {resultados['diabetes']:.4f}")
    print(f"Exactitud del modelo para hipertensión: {resultados['hipertension']:.4f}\n")

Región: 1
Exactitud del modelo para colesterol: 0.7345
Exactitud del modelo para diabetes: 0.8606
Exactitud del modelo para hipertensión: 0.6412

Región: 2
Exactitud del modelo para colesterol: 0.7700
Exactitud del modelo para diabetes: 0.8589
Exactitud del modelo para hipertensión: 0.6679

Región: 3
Exactitud del modelo para colesterol: 0.7660
Exactitud del modelo para diabetes: 0.8771
Exactitud del modelo para hipertensión: 0.6524

Región: 4
Exactitud del modelo para colesterol: 0.8013
Exactitud del modelo para diabetes: 0.8683
Exactitud del modelo para hipertensión: 0.6529

Región: 5
Exactitud del modelo para colesterol: 0.7450
Exactitud del modelo para diabetes: 0.8350
Exactitud del modelo para hipertensión: 0.6300

Región: 6
Exactitud del modelo para colesterol: 0.7456
Exactitud del modelo para diabetes: 0.8417
Exactitud del modelo para hipertensión: 0.6714



In [28]:
 # Obtener la importancia de las características para el modelo de colesterol
importances_col = clf_col.feature_importances_

# Obtener la importancia de las características para el modelo de diabetes
importances_dia = clf_dia.feature_importances_

# Obtener la importancia de las características para el modelo de hipertensión
importances_hip = clf_hip.feature_importances_

In [29]:
for region, df_region in dfs_por_region.items():
    X_region = df_region.iloc[:, 1:-3]  # Ajusta las columnas según tu necesidad
    columnas_originales = X_region.columns
    indices_ordenados = np.argsort(importances_col)[::-1]
    
    print(f"Región: {region}")
    for i in indices_ordenados:
        if importances_col[i]>0:
            print(f"{columnas_originales[i]}: {importances_col[i]}")
    print("\n")

    
    

region_dict = {
    1: 'Metropolitana',
    2: 'Pampeana',
    3: 'Noroeste',
    4: 'Noreste',
    5: 'Cuyo',
    6: 'Patagónica'
}
df['region'] = df['region'].map(region_dict)


    
    
    
    

In [29]:
#columnas_originales = X_region.columns

# Obtener el índice de las características ordenadas por importancia
#indices_ordenados = np.argsort(importances_col)[::-1]

# Mostrar las características ordenadas por importancia
#for i in indices_ordenados:
#    print(f"{columnas_originales[i]}: {importances_col[i]}")

Región: 1
edad: 0.5216316630775409
profesional_indica_sobrepeso: 0.2121814903713719
ingresos: 0.06616529444816927
tiempo_sentado_min_por_dia: 0.05157624453985069
bhch02: 0.03146064776887237
razon_poca_actividad_la_ultima_semana: 0.028091973328504546
altura_cm: 0.0269635915493816
edad_fumo_primera_vez: 0.023768039909959023
considera_su_alimentacion: 0.020790203902949993
tipo_hogar: 0.01737085110339981


Región: 2
edad: 0.5216316630775409
profesional_indica_sobrepeso: 0.2121814903713719
ingresos: 0.06616529444816927
tiempo_sentado_min_por_dia: 0.05157624453985069
bhch02: 0.03146064776887237
razon_poca_actividad_la_ultima_semana: 0.028091973328504546
altura_cm: 0.0269635915493816
edad_fumo_primera_vez: 0.023768039909959023
considera_su_alimentacion: 0.020790203902949993
tipo_hogar: 0.01737085110339981


Región: 3
edad: 0.5216316630775409
profesional_indica_sobrepeso: 0.2121814903713719
ingresos: 0.06616529444816927
tiempo_sentado_min_por_dia: 0.05157624453985069
bhch02: 0.0314606477688723

In [30]:
columnas_originales = X_region.columns

# Obtener el índice de las características ordenadas por importancia
indices_ordenados = np.argsort(importances_dia)[::-1]

# Mostrar las características ordenadas por importancia
for i in indices_ordenados:
    print(f"{columnas_originales[i]}: {importances_dia[i]}")

control_diabetes: 0.3673221048994622
familiares_directos_diabetes: 0.18765304915169528
edad: 0.132251441352402
nivel_instruccion: 0.09321583106808412
peso_kg: 0.04947028595534469
profesional_indica_sobrepeso: 0.041404390487782244
consumido_alcohol: 0.033903072507597845
nivel_actividad_fisica: 0.0302575516697059
familiares_no_directos_diabetes: 0.02856378823656402
consumo_regular_riesgo: 0.02170117061517555
af_fue_traslado: 0.014257314056186167
af_intensa_min_totales_por_semana_nsnc: 0.0
af_intensa_min_totales_por_semana: 0.0
af_intensa_veces_por_semana: 0.0
sexo: 0.0
ansiedad_depresion: 0.0
cobertura_salud_nueva: 0.0
situacion_conyugal: 0.0
asignaciones: 0.0
bhch02: 0.0
af_moderada_min_totales_por_semana: 0.0
ingresos: 0.0
tipo_hogar: 0.0
cant_componentes: 0.0
baño: 0.0
obtiene_agua: 0.0
obtiene_gas: 0.0
tipo_vivienda: 0.0
tamanio_aglomerado: 0.0
af_moderada_veces_por_semana: 0.0
caminata_min_totales_por_semana_nsnc: 0.0
actividad_fisica_moderada_min_totales_por_semana_nsnc: 0.0
alguna

In [31]:
columnas_originales = X_region.columns

# Obtener el índice de las características ordenadas por importancia
indices_ordenados = np.argsort(importances_hip)[::-1]

# Mostrar las características ordenadas por importancia
for i in indices_ordenados:
    print(f"{columnas_originales[i]}: {importances_hip[i]}")

edad: 0.49863757742473047
profesional_indica_sobrepeso: 0.12823577116563586
peso_kg: 0.1249921501514139
control_diabetes: 0.047306954252627736
nivel_instruccion: 0.03225241825249676
biha01: 0.03025278422499971
actualmente_fuma_cigarrillos: 0.027005484070496737
ingresos: 0.025066423518624998
situacion_conyugal: 0.02466401293802959
consumido_alcohol: 0.024118511101140907
ansiedad_depresion: 0.020600151319312622
tiempo_sentado_min_por_dia: 0.016867761580490737
cant_componentes: 0.0
af_moderada_veces_por_semana: 0.0
af_intensa_min_totales_por_semana_nsnc: 0.0
af_intensa_min_totales_por_semana: 0.0
af_intensa_veces_por_semana: 0.0
cobertura_salud_nueva: 0.0
tamanio_aglomerado: 0.0
tipo_vivienda: 0.0
baño: 0.0
sexo: 0.0
bhch02: 0.0
asignaciones: 0.0
actividad_fisica_moderada_min_totales_por_semana_nsnc: 0.0
tipo_hogar: 0.0
obtiene_gas: 0.0
obtiene_agua: 0.0
af_moderada_min_totales_por_semana: 0.0
caminata_min_totales_por_semana_nsnc: 0.0
caminata_veces_por_semana: 0.0
caminata_min_totales_po

In [75]:
    
    
    
    
    # Variables independientes
    
X_region = df_region.iloc[:, 1 :-3]  # Todas las columnas menos las últimas tres y las primeras 2 (i y region)

In [76]:
#variables dependientes, 1 resultado por cada variable

y_region_colesterol = df_region['colesterol_alto']
y_region_diabetes = df_region['dijeron_diabetes']
y_region_hipertension = df_region['veces_presion_alta']

In [77]:
# Dividir los datos en conjuntos de entrenamiento y prueba para cada enfermedad

#25% para prueba y el 75% para modelado. Random_state = numero hace que sienmpre use igual el dataset y el resultado sea igual, si pongo none se 
#varia el datatset y puede dar distintos resultados (mas recomendado en analisis reales)
X_train_col, X_test_col, y_train_col, y_test_col = train_test_split(X_region, y_region_colesterol, test_size=0.25, random_state=0)
X_train_dia, X_test_dia, y_train_dia, y_test_dia = train_test_split(X_region, y_region_diabetes, test_size=0.25, random_state=0)
X_train_hip, X_test_hip, y_train_hip, y_test_hip = train_test_split(X_region, y_region_hipertension, test_size=0.25, random_state=0)


In [78]:
#escalado de variables


#from sklearn.preprocessing import StandardScaler # Importamos la librería para escalar las variables independientes
#sc = StandardScaler() # Creamos el objeto para escalar las variables
#X_train = sc.fit_transform(X_train) # Escalamos las variables de entrenamiento
#X_test = sc.transform(X_test) # Escalamos las variables de test

In [79]:
# Crear y entrenar el modelo de árbol de decisión para cada enfermedad

clf_col = DecisionTreeClassifier(criterion = 'entropy', random_state=0 ,max_depth=4)
clf_col.fit(X_train_col, y_train_col)

clf_dia = DecisionTreeClassifier(criterion = 'entropy', random_state=0 ,max_depth=4)
clf_dia.fit(X_train_dia, y_train_dia)

clf_hip = DecisionTreeClassifier(criterion = 'entropy', random_state=0 ,max_depth=4)
clf_hip.fit(X_train_hip, y_train_hip)


DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)

In [80]:
#pruebo modelo con conjunto de prueba


y_pred_col = clf_col.predict(X_test_col)
y_pred_dia = clf_dia.predict(X_test_dia)
y_pred_hip = clf_hip.predict(X_test_hip)


In [81]:
#exactitud
acc_col = accuracy_score(y_test_col, y_pred_col)
acc_dia = accuracy_score(y_test_dia, y_pred_dia)
acc_hip = accuracy_score(y_test_hip, y_pred_hip)

In [85]:
# Almacenar los modelos y sus resultados
modelos_por_region[region] = {
    'colesterol': clf_col,
    'diabetes': clf_dia,
    'hipertension': clf_hip
}

resultados_por_region[region] = {
    'colesterol': acc_col,
    'diabetes': acc_dia,
    'hipertension': acc_hip
}


In [86]:

# Mostrar los resultados
for region, resultados in resultados_por_region.items():    #resultados_por_region.items():
    print(f"Región: {region}")
    print(f"Exactitud del modelo para colesterol: {resultados['colesterol']:.4f}")
    print(f"Exactitud del modelo para diabetes: {resultados['diabetes']:.4f}")
    print(f"Exactitud del modelo para hipertensión: {resultados['hipertension']:.4f}\n")

Región: 6
Exactitud del modelo para colesterol: 0.7260
Exactitud del modelo para diabetes: 0.8417
Exactitud del modelo para hipertensión: 0.6681

